## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import geopy.distance
import matplotlib.pyplot as plt
from shapely.geometry import Point

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold


from scipy import stats
from sklearn import metrics
import folium

import math

from descartes import PolygonPatch
import altair as alt

from pprint import pprint

# for the notebook only (not for JupyterLab) run this command once per session
alt.renderers.enable('notebook')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
### Consts
datapath = '../rawdata/sensors/'
filename = datapath + 'nodes.txt'

In [ ]:
areas_coords = [
    (40.748203, -74.002728),
    (40.744006, -73.990839),
    (40.739258, -73.978691),
    (40.730250, -73.981419),
    (40.730999, -73.997317),
    (40.736302, -74.003391),
    (40.727475, -74.005537),
    (40.722948, -73.995838),
    (40.719485, -73.984144),
    (40.718380, -74.006466),
    (40.710166, -74.008929),
    (40.714867, -73.993454),
    (40.758490, -73.996376),
    (40.754586, -73.985882),
    (40.750930, -73.976355),
    (40.760797, -73.967662),
    (40.762536, -73.977212),
    (40.765541, -73.986568)]

In [ ]:
map_osm = folium.Map(location=[40.742, -73.956], zoom_start=12, tiles="cartodbpositron")

plotCircleRegions(areas_coords, map_osm)
plotMapPoints(coords, map_osm)

display(map_osm)


In [ ]:



f = open(filename)
sensors = {}
coords=[[]]
for line in f:
    s, lat, lon = line.split(' ')
    
    # collecting sensor metadata
    sensors[s] = {}
    sensors[s]['lat'] = float(lat)
    sensors[s]['lon'] = float(lon) 
    
    
    for circle in areas_coords:
        isWithin = pointWithinCircle([sensors[s]['lat'], sensors[s]['lon']], [circle[0], circle[1], 300])
        
        if(isWithin):
            coords[0].append(Point(sensors[s]['lat'], sensors[s]['lon']))

        

## Helper Functions

### Geospatial Functions

In [ ]:
nyc_boundaries = gpd.read_file('../data/nyc_boundaries/NYC_Boundaries.geojson')

def pointWithinCircle(point, circle):
    ## Return if a given point is within a circle
    c = (circle[0], circle[1])
    r = circle[2]
    dist = geopy.distance.distance(c, point).meters
    if dist <= r:
        return True

    return False

### Visualization Functions

In [ ]:
def plotMapPoints(mapCoordGroups, map_osm):
    
    for group in mapCoordGroups:
        for coord in group:
            folium.CircleMarker(
                location=[coord.x, coord.y],
                radius=5,
                fill=True,
                fill_opacity=0.5,
                fill_color="red",
                color="black").add_to(map_osm)
    
def plotCircleRegions(circleRegions, map_osm): 
    
    for circle in circleRegions: 
        
        circle_str = '('+  str(circle[0]) + ' , ' + str(circle[1]) + ')'
        
        folium.Circle(
            radius=300,
            location=[circle[0], circle[1]],
            popup='The Waterfront',
            color='crimson',
            fill=False,
            tooltip=circle_str).add_to(map_osm)